In [4]:
# Parameters
input_data_path = ""
batch = ""
label = ""

Jupyter notebook for seurat cca integration algoritm

Author: Erno Hänninen

Title: run_seurat_cca.ipynb

Created: 2022-12-06

In [5]:
require(Seurat)

In [ ]:
sobj = readRDS(input_data_path)
sobj

In [7]:
# This function is from https://github.com/theislab/scib-pipeline/blob/0c7be53b1000864fcd31a7b7594f9a5071204233/scripts/integration/integration.R
#runSeuratRPCA = function(data, batch, hvg=2000) {
runSeuratRPCA = function(data, batch, hvg=500) {


	  require(Seurat)
	  batch_list = SplitObject(data, split.by = batch)

          #features <- SelectIntegrationFeatures(batch_list)
          batch_list <- lapply(X = batch_list, FUN = function(x) {
		  x  <- ScaleData(x, features = hvg)
		  x <- RunPCA(x, features = hvg)
		  return(x)
	  })

	  anchors = FindIntegrationAnchors(
	          object.list = batch_list,
	          anchor.features = hvg,
 		  scale = T,
		  l2.norm = T,
		  dims = 1:30,
        	  k.anchor = 5,
        	  k.filter = 200,
        	  k.score = 30,
		  reduction = "rpca",
        	  max.features = 200,
        	  eps = 0)
	  integrated = IntegrateData(
        	   anchorset = anchors,
		   new.assay.name = "integrated",
        	   features = NULL,
        	   features.to.integrate = NULL,
        	   dims = 1:30,
        	   k.weight = 100,
        	   weight.reduction = NULL,
        	   sd.weight = 1,
        	   sample.tree = NULL,
        	   preserve.order = F,
        	   eps = 0,
        	   verbose = T)
	  return(integrated)
}


In [ ]:
if(file.exists(hvg)) {
	hvg<-unlist(readRDS(hvg), use.names=FALSE)
}else {
	hvg <- rownames(sobj@assays$RNA)
}

out = runSeuratRPCA(sobj, batch, hvg)
saveRDS(out, file="../Integrated_adata/seurat_rpca_output.rds")